In [1]:
import numpy as np
import pandas as pd

In [2]:
# Warehouse class object
class Warehouse:
    
    def __init__(self, name):
        '''Constructor method
        :param data: warehouse pandas DataFrame
        :param name: Name of the warehouse to look up
        '''
        self.name = name
#         self.address = data.loc[name,:].Address
#         self.long = data.loc[name,:].Longitude
#         self.lat = data.loc[name,:].Latitude
        self.inventory = {el:0 for el in range(1,13)}
        self.van = 5
        self.delivery_expense = 0
        self.delivery_list = {el:0 for el in customer_regions['Census County Division']}
    
    def updateStock(self,totalInventory):
        inventoryPerBundle = totalInventory/len(bundles)
        for i in bundles:
            self.inventory[i] += inventoryPerBundle
        
    def checkInventory(self,bundle):
        '''
        Returns True if there is inventory, returns false if out of inventory
        '''
        return self.inventory[bundle] != 0 
    
    def reduceInv(self, customer_choice):
        if self.inventory[customer_choice] != 0:
            self.inventory[customer_choice] -= 1
       
    def addOrder(self, customer_region):
        self.delivery_list[customer_region] += 1
    
        
    def deliver(self, distance_df):
        
        # if number of orders to deliver is equal to the size of van
        if sum(self.delivery_list.values()) == self.van:
            
            # loop through the delivery_list dictionary
            for key in self.delivery_list.keys():
                
                # if there is an order or more
                if self.delivery_list[key] != 0: 
                    # get distance from this warehouse to region
                    dist_to_region = distance_df.loc[self.name][key]
                    
                    # calculate delivery expense and add to total
                    self.delivery_expense += dist_to_region * 0.72
                    
                    # reset delivery_list once orders have been fulfilled
                    self.delivery_list[key] = 0
                
    def endOfWeekDelivery(self, distance_df):
        if sum(self.delivery_list.values()) > 0:
            # loop through the delivery_list dictionary
            for key in self.delivery_list.keys():
                
                # if there is an order or more
                if delivery_list[key] != 0: 
                    # get distance from this warehouse to region
                    dist_to_region = distance_df.loc[self.name][key]
                    
                    # calculate delivery expense and add to total
                    self.delivery_expense += dist_to_region * 0.72
                    
                    # reset delivery_list once orders have been fulfilled
                    delivery_list[key] = 0

In [3]:
bundles = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] #these are the 12 bundles that Joymode sells

class Customer:
    def __init__(self):
        self.region = ""
        self.desire = 0
        self.satisfaction = 5
    
    def generateLocation(self, prob, regions):
        '''
        This says where a specific customer is located, randomly generated based on population distribution
        Input: The list of regions and the list of populations in each of those regions
        Return: Which region was selected
        '''
        self.region = np.random.choice(regions, p=prob)
        return self.region

    def generateDesire(self, bundles):
        '''
        This generates a list of a customer top three bundles based on a equal probability of desire for each bundle
        Input: The list of bundles
        Return: The generated list of desires
        '''
        desire =np.random.choice(bundles, 3, replace=False)
        self.desire = list(desire)
        return self.desire
    
    def decreaseSatisfaction(self):
        self.satisfaction -= 2

    def increaseSatisfaction(self,preference):
        '''
        first choice: increase by 2
        second choice: increase by 1
        third choice: nothing 
        '''
        if self.satisfaction <= 8:
            if preference == 0:
                self.satisfaction += 2
            elif preference == 1:
                self.satisfaction += 1
        elif self.satisfaction == 9 and preference in range(0,2):
            self.satisfaction += 1 # since most is 10
            


   

In [4]:
def calculatePopDensity(household, regions):
    total_household = 0
    prob = []
    for i in household:
        total_household += i
    for i in household:
        p = i / total_household
        prob.append(p)
    return prob

In [5]:
import csv
#import Customer from Customer

class Joymode:
    def __init__(self):
        self.warehouses = {}
        self.customers = []
        self.customerslost = 0
        self.customerbase = 120 #3000
        self.profit = 150*self.customerbase        
        self.customersatisfaction = 1

    def createCustomers(self,household,regions): # might have to be edited 
        '''
        This program will create 3000 individual customers, generate their locations, and add them to our warehouse list
        '''
        for i in range(self.customerbase):
            a = Customer()
            a.generateLocation(household, regions)
            self.customers.append(a)


    def createWarehouses(self, warehouselist):
        '''
        This function will populate the warehouse class with the warehouses we were given and add them to our warehouse list
        '''
        for name in warehouselist: # i IS NAME 
            self.warehouses[name]=Warehouse(name)
            
    def getWeekDemand(self,avgCustomer):
        '''
        This picks out randomn number of customers that want to rent on a given week, using exponential distribution. 
        Calculates the number of customers that will order that week, ie. 250
        Randomly selects 250 customers from the all the customers at Joymode. 
        Input: average number of customers that order per week
        Return: list containing customer objects, representing customers that will order that week 
        '''
        # generate number of customers 
        numCustomers = 0
        time = 0
        a = 1/avgCustomer
        while time < 1: 
            time += np.random.exponential(a)
            numCustomers += 1
        numCustomers -= 1 # to subtract the last person, because their time is over 250
        numCustomers
        import random
        # select customers from customer base 
        customerList = random.sample(self.customers, numCustomers)
        return customerList
        
    def getAvgSatisfaction(self):
        '''
        This function gets the satisfaction of every customer and creates an avg customer satisfaction
        Returns: Average Customer Satisfaction
        '''
        for customer in self.customers:
            self.customersatisfaction += customer.getSatiscation()
        self.customersatisfaction /= self.customerbase
        return self.customersatisfaction
    
#     def findClosestTwoWarehouse(self,customerLocation):
#         '''
#         Input: customer's location 
#         Returns: list of the INDEX TWO CLOSEST warehouse to the customer
#         '''
#         # df.loc[customerLocation].sort_values()[0:2] this returns the DISTANCE!!! NOT NAME
#         houseIndex = df2[customerLocation].argsort()[0:2] # THIS RETURNS THE INDEX
#         warehouseName = df2.columns[houseindex].tolist() # THIS RETURNS THE NAME
#         return warehouseName
    
    def checkWarehouses(self,customerLocation,customerChoices):
        '''
        Input: list containing customer's top 3 choices, list containing 2 closest warehouse to customer
        Returns: the customer's choice if it is in stock and NAME of closest warehouse that has item
        '''
        warehouseList = distance[customerLocation].sort_values().index.tolist() 
        # contains name of warehouses, in order of closest distance to customer
        for choice in customerChoices:
            # check warehouse list
            for name in warehouseList:
                # if choice exists in warehouse then return it
                if self.warehouses[name].checkInventory(choice) == True:
                    return  customerChoices.index(choice), choice, name
        return "none","noStock","noStock" # return if none of customer's choices are in stock at 2 closest warehouse
    
    def getNumCust(self):
        return len(self.customers)
    
    def simulateWeek(self,customerList):
        '''
        Input: list containing customer objects for ONE WEEK
        Output: Joymode's profit for the week
        '''
        for cust in customerList:
            preference,bundle,name = self.checkWarehouses(cust.region, cust.desire)
            if bundle != "noStock":
                self.warehouses[name].reduceInv(bundle)
                self.warehouses[name].addOrder(cust.region)
                self.warehouses[name].deliver(distance)
                cust.increaseSatisfaction(preference)
#                     print(str(i)+") "+str(bundle)+", "+  name+", "+cust.region, str(cust.satisfaction))
#                     i+=1
            else:
                cust.decreaseSatisfaction()
                if cust.satisfaction <=1:
                    self.customers.remove(cust)
                #print("NO STOCK", name + ", " + cust.region, str(cust.satisfaction))

            #subtract all warehouses delivery expenses for the week from Joymode's profit
            for warehouse in self.warehouses:
                self.profit -= warehouse.delivery_expense
        return self.profit
    
    def export(self):
        '''
        This function creates a csv file containing the program results
        '''
        with open('joymode_results.csv', mode='w') as joymode_results: ##im not sure how to include inventory levels for each area, is each one a column?
            joymode_results = csv.writer(joymode_results, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            joymode_results.writerow(['Profit', 'Number of Customers', 'Customers Lost', 'Customer Satisfaction'])
            joymode_resultsmode_results.writerow([self.profit, self.customerbase, self.customerslost, self.customersatisfaction])
            

In [6]:
customer_regions = pd.read_csv("customer_regions.csv")
joymode_warehouses = pd.read_csv("joymode_warehouses.csv")
customer_regions.head()

,ID,Census County Division,Population,Households,Latitude,Longitude
0,603790015,Agoura Hills-Malibu,63824,26547,34.059545,-118.755208
1,603790560,Compton,342576,89732,33.869642,-118.245185
2,603790730,Downey-Norwalk,414144,123852,33.902535,-118.109263
3,603790810,East San Gabriel Valley,935875,276360,34.062534,-117.858474
4,603791400,Inglewood,369694,125450,33.922789,-118.335092


In [7]:
customer_regions["Population"].tolist()

[63824,
 342576,
 414144,
 935875,
 369694,
 569100,
 2504251,
 91365,
 268040,
 1771545,
 89736,
 138043,
 501237,
 318982,
 145438,
 323899,
 316911]

In [8]:
joymode_warehouses['Name'].tolist()

['Joymode Downtown LA',
 'Joymode Culver City',
 'Joymode South Pasadena',
 'Joymode Tarzana',
 'Joymode Long Beach']

### CREATE JOYMODE AND CUSTOMERS

In [9]:
test = Joymode()
prob = calculatePopDensity(customer_regions["Households"].tolist(),customer_regions["Census County Division"].tolist())
test.createCustomers(prob,customer_regions["Census County Division"].tolist())

In [10]:
# for i in test.customers:
#     print(i.region)

### create warehouses

In [11]:
test.createWarehouses(joymode_warehouses['Name'].tolist())
test.warehouses

{'Joymode Downtown LA': <__main__.Warehouse at 0x121bd9b38>,
 'Joymode Culver City': <__main__.Warehouse at 0x121bd9ba8>,
 'Joymode South Pasadena': <__main__.Warehouse at 0x121bd9be0>,
 'Joymode Tarzana': <__main__.Warehouse at 0x121bd9c18>,
 'Joymode Long Beach': <__main__.Warehouse at 0x121bd9c50>}

### create customer demand for week

In [12]:
customerList = test.getWeekDemand(65)
for cust in customerList:
    cust.generateDesire(bundles)

### start of simulate week function?
### find nearest warehouse that has item

In [13]:
distance = pd.read_csv("distance.csv")
distance.rename(columns={'Unnamed: 0':'Warehouse'}, inplace=True)
distance.set_index("Warehouse",inplace=True)
distance.head()

,Agoura Hills-Malibu,Compton,Downey-Norwalk,East San Gabriel Valley,Inglewood,Long Beach-Lakewood,Los Angeles,Palos Verdes,Pasadena,San Fernando Valley,Santa Monica,South Bay Cities,South Gate-East Los Angeles,Southwest San Gabriel Valley,Torrance,Upper San Gabriel Valley,Whittier
Warehouse,,,,,,,,,,,,,,,,,
Joymode Downtown LA,34.4,15.2,17.8,26.4,12.0,24.1,9.5,26.7,13.5,21.6,15.0,20.6,9.9,12.4,20.9,18.6,18.6
Joymode Culver City,26.1,22.3,26.7,34.4,13.0,29.3,4.0,25.9,20.7,24.0,6.7,17.1,16.2,18.7,18.4,26.6,24.8
Joymode South Pasadena,42.8,26.0,22.3,20.8,20.7,30.7,17.9,35.4,4.4,21.4,23.4,29.3,14.5,5.7,29.5,12.4,23.1
Joymode Tarzana,28.4,36.5,40.9,46.2,27.2,43.5,23.2,40.1,26.9,17.6,20.4,31.3,30.7,32.2,32.6,35.3,39.3
Joymode Long Beach,50.1,11.8,15.8,37.0,18.2,2.6,23.8,13.1,34.4,43.0,30.7,17.7,18.2,25.0,16.7,33.1,22.4


In [14]:
# for simulation , there would be for loop her and replace 12 with stock
for warehouse in test.warehouses.values():
    warehouse.updateStock(12)    



In [15]:
# for cust in customerList:
#     if test.checkWarehouses(cust.region, cust.desire) == "noStock":
#         print("NO STOCK",name+", "+cust.region)
#     else:
#         bundle,name = test.checkWarehouses(cust.region, cust.desire)
        
        
        
#         test.warehouses[name].reduceInv(bundle)
#         print(bundle,name+", "+cust.region)
i= 0

year = 52

    
    if week%4==0:
        # add profits from membership fee
        test.profit += test.getNumCust()*membershipFee
    if week%12 == 0:
        # get how many customers are in joymode
        # see how many tens
        # 
        
    print(test.warehouses['Joymode Tarzana'].delivery_list)  
membershipFee = 150

0) 9, Joymode South Pasadena, Southwest San Gabriel Valley 7
1) 6, Joymode Downtown LA, Inglewood 7
2) 4, Joymode Culver City, Santa Monica 7
3) 6, Joymode South Pasadena, East San Gabriel Valley 7
4) 5, Joymode Tarzana, San Fernando Valley 7
5) 11, Joymode Culver City, Santa Monica 7
6) 2, Joymode Tarzana, San Fernando Valley 7
7) 11, Joymode Long Beach, South Bay Cities 7
8) 11, Joymode Downtown LA, Los Angeles 7
9) 12, Joymode Downtown LA, South Gate-East Los Angeles 7
10) 10, Joymode Downtown LA, South Gate-East Los Angeles 7
11) 2, Joymode Culver City, Los Angeles 7
12) 11, Joymode South Pasadena, South Gate-East Los Angeles 7
13) 2, Joymode Downtown LA, Whittier 7
14) 1, Joymode Culver City, Los Angeles 7
15) 10, Joymode Tarzana, San Fernando Valley 7
16) 9, Joymode Culver City, Los Angeles 7
17) 3, Joymode Culver City, Los Angeles 7
18) 5, Joymode Culver City, Los Angeles 7
19) 7, Joymode Culver City, Los Angeles 7
20) 7, Joymode Downtown LA, Los Angeles 7
21) 4, Joymode Downtow